In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)
driver.get("https://www.rozee.pk/job/jsearch/q/Software%20Engineer")
time.sleep(5)

job_list = []

while len(job_list) < 40:
    jobs = driver.find_elements(By.CLASS_NAME, "job")

    for job in jobs:
        if len(job_list) >= 40:
            break
        try:
            job_link = job.find_element(By.XPATH, ".//a").get_attribute("href")
            driver.get(job_link)
            time.sleep(3)

            title = driver.find_element(By.XPATH, ".//h1[@class='jtitle']//bdi").text

            company_elements = driver.find_elements(By.XPATH, ".//h2[@class='cname']//bdi")
            company = company_elements[0].text if company_elements else "N/A"

            job_location_elements = driver.find_elements(By.XPATH, "//a[contains(@href, 'jobs-in-')][contains(@class, 'jblk')]")
            job_locations = [elem.text for elem in job_location_elements]
            job_location = job_locations[0] if len(job_locations) == 1 else ", ".join(job_locations) if job_locations else "N/A"

            job_type_elements = driver.find_elements(By.XPATH, "//a[contains(@href, 'full-time-jobs-in-pakistan') or contains(@href, 'permanent-jobs-in-pakistan')][contains(@class, 'jblk')]")
            job_type = ", ".join([elem.text for elem in job_type_elements]) if job_type_elements else "N/A"

            skills_section_elements = driver.find_elements(By.XPATH, "//h4[contains(text(), 'Skills')]/following-sibling::div[contains(@class, 'jcnt')]")
            if skills_section_elements:
                skills_section = skills_section_elements[0]
                skill_elements = skills_section.find_elements(By.TAG_NAME, "a")
                skills = [skill.text.strip() for skill in skill_elements if skill.text.strip()]
            else:
                skills = ["N/A"]

            job_description_elements = driver.find_elements(By.XPATH, "//div[@dir='ltr']")
            if job_description_elements:
                job_description_container = job_description_elements[0]
                job_description_parts = job_description_container.find_elements(By.XPATH, ".//*")
                job_description = """\n""".join([part.text.strip() for part in job_description_parts if part.text.strip()])
                job_description = f'"""{job_description}"""' if job_description else '"""N/A"""'
            else:
                job_description = '"""N/A"""'

            job_list.append({
                "Job Title": title,
                "Company Name": company,
                "Location": job_location,
                "Required Skills": skills,
                "Job Type": job_type,
                "Job Description": job_description
            })

            driver.execute_script("window.history.go(-1)")
            time.sleep(3)

        except Exception as e:
            print("Error extracting job information:", e)

    next_buttons = driver.find_elements(By.LINK_TEXT, "Next")
    if next_buttons:
        next_buttons[0].click()
        time.sleep(5)
    else:
        break


driver.quit()

job_data = pd.DataFrame(job_list)
print(job_data)


Error extracting job information: Message: no such element: Unable to locate element: {"method":"xpath","selector":".//a"}
  (Session info: chrome=132.0.6834.197); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00AC74A3+25091]
	(No symbol) [0x00A4DC04]
	(No symbol) [0x0092B373]
	(No symbol) [0x0096F4DC]
	(No symbol) [0x0096F65B]
	(No symbol) [0x00964F21]
	(No symbol) [0x00991F54]
	(No symbol) [0x00964E44]
	(No symbol) [0x009921A4]
	(No symbol) [0x009AB49E]
	(No symbol) [0x00991CA6]
	(No symbol) [0x009631D5]
	(No symbol) [0x0096435D]
	GetHandleVerifier [0x00DC07C3+3142947]
	GetHandleVerifier [0x00DD1A2B+3213195]
	GetHandleVerifier [0x00DCC412+3191154]
	GetHandleVerifier [0x00B68720+685184]
	(No symbol) [0x00A56E1D]
	(No symbol) [0x00A53E18]
	(No symbol) [0x00A53FB6]
	(No symbol) [0x00A466F0]
	BaseThreadInitThunk [0x75667BA9+25]
	RtlInitializeExceptionChain 

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd

# Initialize WebDriver
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)
driver.get("https://www.almas.pk/")
time.sleep(3)  # Allow page to load

women_link = driver.find_element(By.XPATH, '//a[@href="/collections/women-apparel"]')
women_link.click()  # Directly click the link
time.sleep(5)  # Wait for navigation
product_names = []
product_prices = []

product_links = []
image_links = []

last_height = driver.execute_script("return document.body.scrollHeight")
while True:
    # Scroll down to the bottom of the page
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(5)  # Wait for new products to load

    # Check for new products
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:  # If no new content loaded, break the loop
        break
    last_height = new_height

# Step 3: Find all product containers using div.card
products = driver.find_elements(By.CLASS_NAME, "card")
for product in products[:5]:
    if not product.is_displayed():  # Skip products that are not visible
        continue
    # Locate the product name and link from the anchor tag inside div.card-product
    link_element = product.find_element(By.CSS_SELECTOR, "a.card-link")

    product_name = link_element.get_attribute("title")
    product_link = "https://www.almas.pk" + link_element.get_attribute("href")
    

 
    image = product.find_element(By.TAG_NAME, "img").get_attribute("src")
    
    try:
        # Check for sale price first
        sale_price = product.find_element(By.CLASS_NAME, "price-item--sale").text.strip()
    except Exception:
        sale_price = None

    try:
        # Check for regular price if no sale price is available
        if sale_price:
            price = sale_price
        else:
            price = product.find_element(By.CLASS_NAME, "price-item--regular").text.strip()
    except Exception:
            # If no regular price is available, check for another price structure
            try:
                price = product.find_element(By.CLASS_NAME, "price__regular").text.strip()
            except Exception:
                price = "Price not available"

        # Store collected data
    product_names.append(product_name)
    product_prices.append(price)
    product_links.append(product_link)
    image_links.append(image)
    
    #print(product_name,"  ", image)
    
Final=pd.DataFrame({"Product Title": product_names,
                    "Price": product_prices,
                    "Links":product_links,
                    "image":image_links})



In [14]:
Final

,Product Title,Price,Links,image
0,ALMAS SLEEVELESS KNIT SWEATER-BEIGE,"Rs.1,920",https://www.almas.pkhttps://www.almas.pk/produ...,https://www.almas.pk/cdn/shop/files/BE032-Beig...
1,ALMAS SLEEVELESS KNIT SWEATER-BLACK,"Rs.1,920",https://www.almas.pkhttps://www.almas.pk/produ...,https://www.almas.pk/cdn/shop/files/BE032-Blac...
2,ALMAS SLEEVELESS KNIT SWEATER-NAVY,"Rs.1,920",https://www.almas.pkhttps://www.almas.pk/produ...,https://www.almas.pk/cdn/shop/files/BE032-Navy...
3,ATHELEISURE STRIPE SWEATSHIRT-BLACK,"Rs.1,920",https://www.almas.pkhttps://www.almas.pk/produ...,https://www.almas.pk/cdn/shop/files/STW62_6_53...
4,ATHELEISURE STRIPE SWEATSHIRT-DENIM,"Rs.1,920",https://www.almas.pkhttps://www.almas.pk/produ...,https://www.almas.pk/cdn/shop/files/STW62_2_53...
...,...,...,...,...
168,WOMEN JOGGER PANT-BLACK-GREEN-RED,"Rs.1,920",https://www.almas.pkhttps://www.almas.pk/produ...,https://www.almas.pk/cdn/shop/files/stw03-1bla...
169,WOOL JACKET-BLACK,"Rs.7,200",https://www.almas.pkhttps://www.almas.pk/produ...,https://www.almas.pk/cdn/shop/files/CL06-Black...
170,WOOL JACKET-CAMEL,"Rs.7,200",https://www.almas.pkhttps://www.almas.pk/produ...,https://www.almas.pk/cdn/shop/files/CL06_6_533...
171,WOOL JACKET-GREEN,"Rs.7,200",https://www.almas.pkhttps://www.almas.pk/produ...,https://www.almas.pk/cdn/shop/files/CL06_2_ee8...


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)
driver.get("https://www.youtube.com/results?search_query=Deepseek&sp=EgIIBQ%253D%253D")
time.sleep(3)

for _ in range(3):
    driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END)
    time.sleep(2)

videos = driver.find_elements(By.XPATH, "//ytd-video-renderer")
video_data = []

for video in videos[:5]:
    try:
        title = video.find_element(By.ID, "video-title").text
        #channel = video.find_element(By.XPATH, ".//yt-formatted-string[@id='text']").text
        channel = video.find_element(By.CSS_SELECTOR, "div.cbox h3.reel-player-header-channel-title").text
        views = video.find_element(By.XPATH, ".//span[contains(text(), 'views')]").text
        upload_date = video.find_element(By.XPATH, ".//div[@id='metadata-line']/span[2]").text
        
        video_url = video.find_element(By.ID, "video-title").get_attribute("href")
        driver.execute_script("window.open(arguments[0], '_blank');", video_url)
        driver.switch_to.window(driver.window_handles[1])
        time.sleep(3)
        
        try:
            duration = driver.find_element(By.CLASS_NAME, "ytp-time-duration").text
        except:
            duration = "N/A"
        
        try:
            description = driver.find_element(By.ID, "description").text
        except:
            description = "No description"
        
        driver.close()
        driver.switch_to.window(driver.window_handles[0])
        video_data.append([title, channel, views, upload_date, duration, description])
    except:
        continue
driver.quit()

video_df = pd.DataFrame(video_data, columns=["Title", "Channel", "Views", "Upload Date", "Duration", "Description"])
video_df.to_csv("youtube_videos.csv", index=False)

video_df[['Views_Num', 'Views_Suffix']] = video_df['Views'].str.extract(r'([\d\.]+)([KMB]?)')
suffix_mapper = {'': 1,'K': 1000, 'M': 1000000, 'B': 1000000000}
video_df['Views_Total'] = video_df['Views_Num'].astype(float) * video_df['Views_Suffix'].map(suffix_mapper)

most_viewed_video = video_df.iloc[video_df['Views_Total'].idxmax()]
print("Most viewed video:", most_viewed_video)

avg_duration = video_df['Duration'].str.extract('(\d+):(\d+)').astype(float).fillna(0).apply(lambda x: x[0] * 60 + x[1], axis=1).mean()
print("Average video duration (seconds):", avg_duration)


<>:67: SyntaxWarning: invalid escape sequence '\d'
<>:68: SyntaxWarning: invalid escape sequence '\d'
<>:67: SyntaxWarning: invalid escape sequence '\d'
<>:68: SyntaxWarning: invalid escape sequence '\d'
C:\Users\Safwan\AppData\Local\Temp\ipykernel_3932\3676124806.py:67: SyntaxWarning: invalid escape sequence '\d'
  most_viewed_video = df.iloc[df['Views'].str.replace(',', '').str.extract('(\d+)').astype(float).idxmax()]
C:\Users\Safwan\AppData\Local\Temp\ipykernel_3932\3676124806.py:68: SyntaxWarning: invalid escape sequence '\d'
  avg_duration = df['Duration'].str.extract('(\d+):(\d+)').astype(float).fillna(0).apply(lambda x: x[0] * 60 + x[1], axis=1).mean()


Most viewed video:                                                Title Channel       Views  \
1  How A Chinese Villager Shook Silicon Valley [D...          399K views   

  Upload Date Duration Description  
1  4 days ago     0:25              
Average video duration (seconds): 251.6


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)
website_link = "https://www.lamaretail.com/"
driver.get(website_link)
time.sleep(3)
retail_data = []

women_link = website_link + "collections/woman"
driver.get(women_link)
time.sleep(3)

products = driver.find_elements(By.CSS_SELECTOR, "grid-product__content")
for product in products[:5]:
    product_name = product.find_element(By.CLASS_NAME, "grid-product__title").text
    product_price = product.find_element(By.CLASS_NAME, "grid-product__price").text
    retail_data.append([product_name, product_price, "Women"])

driver.back()
time.sleep(3)
men_link = website_link + "collections/man"
driver.get(men_link)
time.sleep(3)

products = driver.find_elements(By.CSS_SELECTOR, "grid-product__content")
for product in products[:5]:
    product_name = product.find_element(By.CLASS_NAME, "grid-product__title").text
    product_price = product.find_element(By.CLASS_NAME, "grid-product__price").text
    retail_data.append([product_name, product_price, "Men"])

driver.back()
time.sleep(3)
shoes_link = website_link + "collections/shoes"
driver.get(shoes_link)
time.sleep(3)

products = driver.find_elements(By.CSS_SELECTOR, "grid-product__content")
for product in products[:5]:
    product_name = product.find_element(By.CLASS_NAME, "grid-product__title").text
    product_price = product.find_element(By.CLASS_NAME, "grid-product__price").text
    retail_data.append([product_name, product_price, "Shoes"])

driver.back()
time.sleep(3)
accessories_link = website_link + "collections/accessories"
driver.get(accessories_link)
time.sleep(3)

products = driver.find_elements(By.CSS_SELECTOR, "grid-product__content")
for product in products[:5]:
    product_name = product.find_element(By.CLASS_NAME, "grid-product__title").text
    product_price = product.find_element(By.CLASS_NAME, "grid-product__price").text
    retail_data.append([product_name, product_price, "Accessories"])

driver.back()
time.sleep(3)
selfcare_link = website_link + "collections/self-care"
driver.get(selfcare_link)
time.sleep(3)

products = driver.find_elements(By.CSS_SELECTOR, "grid-product__content")
for product in products[:5]:
    product_name = product.find_element(By.CLASS_NAME, "grid-product__title").text
    product_price = product.find_element(By.CLASS_NAME, "grid-product__price").text
    retail_data.append([product_name, product_price, "Self-Care"])

driver.back()
time.sleep(3)
fragrance_link = website_link + "collections/fragrance"
driver.get(fragrance_link)
time.sleep(3)

products = driver.find_elements(By.CSS_SELECTOR, "grid-product__content")
for product in products[:5]:
    product_name = product.find_element(By.CLASS_NAME, "grid-product__title").text
    product_price = product.find_element(By.CLASS_NAME, "grid-product__price").text
    retail_data.append([product_name, product_price, "Fragrance"])

driver.quit()
retail_df = pd.DataFrame(retail_data, columns=["Name", "Price", "Category"])

In [22]:
retail_df

,Name,Price,Category


In [ ]:
last_height = driver.execute_script("return document.body.scrollHeight")
while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(5)
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height